# Lab 8: Deep Research with NASA Technical Reports

Use **o3-deep-research** with **function calling** to research NASA NTRS documents!

> ✅ **Azure OpenAI Compatible**
> 
> This notebook uses the **Chat Completions API with function calling** instead of MCP.
> This approach works with Azure today.

## What is o3-deep-research?

The `o3-deep-research` model is designed for **advanced research tasks**. It can:
- Browse, analyze, and synthesize information from **hundreds of sources**
- Produce **comprehensive, citation-rich reports**
- Use **multi-step reasoning** with tools
- Run **code for complex analysis**

| Standard Chat | o3-deep-research |
|--------------|------------------|
| Single model call | Multi-step reasoning with tool use |
| No tool loops | **Agentic loop** with search/fetch |
| Instant response | Extended reasoning for complex queries |
| Generic answers | **Cited, sourced** research reports |

## Data Source: NASA NTRS

We download **NASA's Technical Reports Server (NTRS)** public metadata:
- 800,000+ NASA technical documents
- Original mission reports, scientific papers, technical memoranda
- Filter for specific topics (default: **Apollo 14**)
- Index into **Foundry IQ** knowledge base

## Prerequisites
- Completed **Lab 1a** (Landing Zone) - for embeddings and APIM gateway
- `.env` file with APIM_URL, APIM_KEY, AI_ENDPOINT

## Step 1: Install Dependencies

In [ ]:
!pip install pandas requests azure-identity azure-search-documents pypdf matplotlib ijson openai -q

## Step 2: Configuration

In [ ]:
import subprocess
import os
import json
import gzip
import time
import requests
import pandas as pd
from pathlib import Path
from dataclasses import dataclass, field, asdict
from typing import Optional, Dict, List, Any
from IPython.display import display, Markdown, HTML, clear_output
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# =============================================================================
# CONFIGURATION - Modify these for your use case
# =============================================================================

SEARCH_TERM = "Apollo 14"      # What to research
MAX_PDF_SIZE_MB = 100          # Maximum total PDF download size
DEEP_RESEARCH_MODEL = "o3-deep-research"
MAX_RESEARCH_ITERATIONS = 10   # Maximum tool call iterations

# Resource names
RG = "deep-research-lab"
LOCATION = "eastus2"
HUB_RG = "foundry-lz-parent"   # Landing Zone resource group from Lab 1a

# =============================================================================
# Load environment from Lab 1a
# =============================================================================

env_path = Path("../.env")
if env_path.exists():
    for line in env_path.read_text().splitlines():
        if '=' in line and not line.startswith('#'):
            key, value = line.split('=', 1)
            os.environ[key.strip()] = value.strip()

APIM_URL = os.environ.get("APIM_URL", "")
APIM_KEY = os.environ.get("APIM_KEY", "")
AI_ENDPOINT = os.environ.get("AI_ENDPOINT", "")
MODEL_NAME = os.environ.get("MODEL_NAME", "gpt-4.1-mini")
EMBEDDING_MODEL = os.environ.get("EMBEDDING_MODEL", "text-embedding-3-large")

# Extract hub account name and APIM name from endpoints
import re
HUB_ACCOUNT_NAME = ""
APIM_NAME = ""
if AI_ENDPOINT:
    match = re.match(r'https://([^.]+)\.', AI_ENDPOINT)
    if match:
        HUB_ACCOUNT_NAME = match.group(1)
if APIM_URL:
    match = re.match(r'https://([^.]+)\.', APIM_URL)
    if match:
        APIM_NAME = match.group(1)

# Get current user
PRINCIPAL_ID = subprocess.run(
    'az ad signed-in-user show --query id -o tsv',
    shell=True, capture_output=True, text=True
).stdout.strip()

# Verify configuration
print("=" * 60)
print("CONFIGURATION STATUS")
print("=" * 60)

if not APIM_URL or not APIM_KEY:
    print("⚠️  [WARN] Missing APIM_URL or APIM_KEY in .env file!")
    print("         Please complete Lab 1a first")
else:
    print("✅ [OK] Azure Configuration Loaded")
    print(f"       APIM Gateway:  {APIM_URL[:50]}...")
    print(f"       APIM Name:     {APIM_NAME}")
    print(f"       Hub Account:   {HUB_ACCOUNT_NAME}")

print()
print(f"📚 Research Term: {SEARCH_TERM}")
print(f"🤖 Model: {DEEP_RESEARCH_MODEL}")
print("=" * 60)

## Step 3: Deploy Infrastructure

This creates:
- **Norway East Hub**: o3-deep-research deployment (region-specific)
- **APIM Backend**: Routes chat completions to Norway East
- **Azure AI Search**: For Foundry IQ knowledge bases

⏱️ Takes ~5-7 minutes

In [78]:
!az group create -n "{RG}" -l "{LOCATION}" -o table

Location    Name
----------  -----------------
eastus2     deep-research-lab


In [ ]:
# Deploy spoke resources with Norway East hub for o3-deep-research
!az deployment group create -g "{RG}" --template-file spoke.bicep \
    -p deployerPrincipalId="{PRINCIPAL_ID}" \
    -p hubResourceGroup="{HUB_RG}" \
    -p hubAccountName="{HUB_ACCOUNT_NAME}" \
    -p apimName="{APIM_NAME}" \
    -o table

In [ ]:
# Get deployment outputs
outputs = json.loads(subprocess.run(
    f'az deployment group show -g "{RG}" -n spoke --query properties.outputs -o json',
    shell=True, capture_output=True, text=True
).stdout)

SEARCH_ENDPOINT = outputs['searchEndpoint']['value']
SEARCH_NAME = outputs['searchName']['value']
NORWAYEAST_HUB_ENDPOINT = outputs['norwayeastHubEndpoint']['value']
NORWAYEAST_HUB_NAME = outputs['norwayeastHubName']['value']

print(f"✅ Deployment Complete!")
print(f"   Search Service:     {SEARCH_NAME}")
print(f"   Search Endpoint:    {SEARCH_ENDPOINT}")
print(f"   Norway East Hub:    {NORWAYEAST_HUB_NAME}")
print(f"   Norway East Endpoint: {NORWAYEAST_HUB_ENDPOINT}")

In [81]:
# Wait for RBAC propagation
for i in range(60, 0, -10):
    clear_output(wait=True)
    print(f"⏳ Waiting for RBAC to propagate... {i}s")
    time.sleep(10)
clear_output(wait=True)
print("✅ RBAC permissions ready!")

✅ RBAC permissions ready!


## Step 4: Download NASA NTRS Metadata

Download the full NASA Technical Reports Server metadata (~2GB compressed).

In [82]:
# Configuration
NTRS_METADATA_URL = "https://www.sti.nasa.gov/docs/ntrs-public-metadata.json.gz"
DATA_DIR = Path("./ntrs_data")
METADATA_FILE = DATA_DIR / "ntrs-public-metadata.json.gz"
PDF_DIR = DATA_DIR / "pdfs"

DATA_DIR.mkdir(exist_ok=True)
PDF_DIR.mkdir(exist_ok=True)

# Download metadata if not present
if not METADATA_FILE.exists():
    print(f"📥 Downloading NTRS metadata (~2GB compressed)...")
    !curl -L -o "{METADATA_FILE}" "{NTRS_METADATA_URL}" --progress-bar
    print(f"✅ Downloaded to {METADATA_FILE}")
else:
    size_mb = METADATA_FILE.stat().st_size / (1024 * 1024)
    print(f"✅ Metadata already exists: {METADATA_FILE} ({size_mb:.1f} MB)")

✅ Metadata already exists: ntrs_data/ntrs-public-metadata.json.gz (398.9 MB)


## Step 5: Filter Documents by Search Term

In [83]:
import ijson

print(f"🔍 Streaming metadata for '{SEARCH_TERM}'...")

search_lower = SEARCH_TERM.lower()
filtered_docs = []
doc_count = 0

with gzip.open(METADATA_FILE, 'rb') as f:
    for doc_id, doc in ijson.kvitems(f, ''):
        doc_count += 1
        if doc_count % 50000 == 0:
            print(f"   Scanned {doc_count:,} docs, found {len(filtered_docs)} matches...", end='\r')
        
        title = doc.get('title', '') or ''
        abstract = doc.get('abstract', '') or ''
        
        if search_lower in title.lower() or search_lower in abstract.lower():
            doc['ntrs_id'] = doc_id
            filtered_docs.append(doc)

print(f"\n✅ Found {len(filtered_docs)} documents matching '{SEARCH_TERM}'")

if filtered_docs:
    df = pd.DataFrame([{
        "id": d.get('ntrs_id'),
        "title": d.get('title', 'N/A')[:60],
        "year": d.get('publications', [{}])[0].get('publicationDate', '')[:4] if d.get('publications') else 'N/A',
    } for d in filtered_docs[:10]])
    display(df)

🔍 Streaming metadata for 'Apollo 14'...
   Scanned 550,000 docs, found 451 matches...
✅ Found 461 documents matching 'Apollo 14'


,id,title,year
0,19740072936,Apollo 14 (mission H-3) Baseline Mission Profile,1969
1,19710001731,"Apollo operations handbook, extravehicular mob...",1970
2,19710004668,Apollo/Saturn 5 consolidated instrumentation p...,1970
3,19710005842,Apollo 14 /AS-509/ operational trajectory for ...,1970
4,19710010322,Apollo 14 laser ranging retro-reflector experi...,1970
5,19950023897,Apollo 14: Shepard Hitting Golf Ball on Moon,1970
6,19710021481,Preliminary geologic investigations of the Apo...,1971
7,19710021482,Soil mechanics experiment,1971
8,19710021486,Suprathermal ion detector experiment /lunar io...,1971
9,19710027929,"Apollo 14 mission, 5 day report",1971


## Step 6: Download PDFs

In [84]:
NTRS_BASE = "https://ntrs.nasa.gov"
NTRS_API_BASE = f"{NTRS_BASE}/api"

def get_downloads(ntrs_id: str) -> list:
    """Get available PDF downloads for a document."""
    try:
        resp = requests.get(f"{NTRS_API_BASE}/citations/{ntrs_id}", timeout=30)
        if resp.ok:
            return [d for d in resp.json().get('downloads', []) 
                    if d.get('mimetype', '').lower() == 'application/pdf']
    except: 
        pass
    return []

def download_pdf(ntrs_id: str, dl: dict) -> tuple:
    """Download a PDF and return (filepath, size)."""
    try:
        rel_path = dl.get('links', {}).get('pdf')
        if rel_path:
            pdf_url = f"{NTRS_BASE}{rel_path}"
        else:
            pdf_url = f"{NTRS_API_BASE}/citations/{ntrs_id}/downloads/{dl.get('name', ntrs_id + '.pdf')}"
        
        filepath = PDF_DIR / f"{ntrs_id}_{dl.get('name', 'doc.pdf').replace('.pdf', '')}.pdf"
        if filepath.exists():
            return filepath, filepath.stat().st_size
        
        resp = requests.get(pdf_url, timeout=120, stream=True)
        if resp.ok and 'application/pdf' in resp.headers.get('content-type', ''):
            with open(filepath, 'wb') as f:
                for chunk in resp.iter_content(8192):
                    f.write(chunk)
            return filepath, filepath.stat().st_size
    except:
        pass
    return None, 0

print(f"📥 Downloading PDFs (max {MAX_PDF_SIZE_MB} MB total)...")
downloaded = []
total_bytes = 0

for doc in filtered_docs:
    if total_bytes >= MAX_PDF_SIZE_MB * 1024 * 1024:
        break
    
    ntrs_id = doc['ntrs_id']
    title = doc.get('title', 'N/A')[:40]
    print(f"  {ntrs_id}: {title}...", end=" ")
    
    downloads = get_downloads(ntrs_id)
    if not downloads:
        print("(no PDF)")
        continue
    
    filepath, sz = download_pdf(ntrs_id, downloads[0])
    if filepath:
        total_bytes += sz
        downloaded.append({
            'ntrs_id': ntrs_id,
            'title': doc.get('title', ''),
            'abstract': doc.get('abstract', ''),
            'authors': [a.get('name', '') for a in doc.get('authorAffiliations', [])],
            'year': doc.get('publications', [{}])[0].get('publicationDate', '')[:4] if doc.get('publications') else '',
            'filepath': filepath
        })
        print(f"✅ ({sz/1024:.1f} KB) [{total_bytes/1024/1024:.1f}/{MAX_PDF_SIZE_MB} MB]")
    else:
        print("(error)")
    time.sleep(0.3)

print(f"\n✅ Downloaded {len(downloaded)} PDFs ({total_bytes/1024/1024:.1f} MB)")

📥 Downloading PDFs (max 100 MB total)...
  19740072936: Apollo 14 (mission H-3) Baseline Mission... ✅ (810.9 KB) [0.8/100 MB]
  19710001731: Apollo operations handbook, extravehicul... ✅ (7576.6 KB) [8.2/100 MB]
  19710004668: Apollo/Saturn 5 consolidated instrumenta... ✅ (3098.6 KB) [11.2/100 MB]
  19710005842: Apollo 14 /AS-509/ operational trajector... ✅ (22696.0 KB) [33.4/100 MB]
  19710010322: Apollo 14 laser ranging retro-reflector ... ✅ (19579.8 KB) [52.5/100 MB]
  19950023897: Apollo 14: Shepard Hitting Golf Ball on ... (no PDF)
  19710021481: Preliminary geologic investigations of t... (no PDF)
  19710021482: Soil mechanics experiment... (no PDF)
  19710021486: Suprathermal ion detector experiment /lu... (no PDF)
  19710027929: Apollo 14 mission, 5 day report... ✅ (1119.4 KB) [53.6/100 MB]
  19710055248: Ar 40/Ar 39 ages from Fra Mauro... (no PDF)
  19710058684: Chemical composition of Apollo 14 soils ... (no PDF)
  19710061141: Ages of crystalline rocks from Fra Mauro... (no 

## Step 7: Extract Text from PDFs

In [85]:
from pypdf import PdfReader

def extract_text(filepath: Path, max_pages: int = 50) -> str:
    """Extract text from PDF."""
    try:
        reader = PdfReader(filepath)
        return "\n\n".join([p.extract_text() or "" for p in reader.pages[:max_pages]])
    except Exception as e:
        print(f"  Error: {e}")
        return ""

print("📄 Extracting text from PDFs...")
documents = []

for doc in downloaded:
    print(f"  {doc['filepath'].name}...", end=" ")
    text = extract_text(doc['filepath'])
    if text and len(text) > 100:
        full_text = f"""Title: {doc['title']}
Authors: {', '.join(doc['authors'])}
Year: {doc['year']}

Abstract:
{doc['abstract']}

Full Text:
{text}"""
        documents.append({
            'ntrs_id': doc['ntrs_id'], 
            'title': doc['title'], 
            'text': full_text, 
            'year': doc['year'], 
        })
        print(f"✅ ({len(text):,} chars)")
    else:
        print("-")

print(f"\n✅ Extracted text from {len(documents)} documents")

📄 Extracting text from PDFs...
  19740072936_19740072936.pdf... ✅ (22,884 chars)
  19710001731_19710001731.pdf... ✅ (39,142 chars)
  19710004668_19710004668.pdf... ✅ (37,973 chars)
  19710005842_19710005842.pdf... ✅ (53,520 chars)
  19710010322_19710010322.pdf... ✅ (35,832 chars)
  19710027929_19710027929.pdf... ✅ (38,249 chars)
  19720007220_19720007220.pdf... ✅ (72,354 chars)
  19720010767_19720010767.pdf... ✅ (35,262 chars)

✅ Extracted text from 8 documents


## Step 8: Create Azure AI Search Index

In [86]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex, SearchField, SearchFieldDataType,
    SemanticConfiguration, SemanticField, SemanticPrioritizedFields, SemanticSearch,
    VectorSearch, HnswAlgorithmConfiguration, VectorSearchProfile
)
from azure.search.documents import SearchIndexingBufferedSender
from azure.core.exceptions import HttpResponseError

credential = DefaultAzureCredential()
index_client = SearchIndexClient(endpoint=SEARCH_ENDPOINT, credential=credential)

INDEX_NAME = f"{SEARCH_TERM.lower().replace(' ', '-')}-research"

index = SearchIndex(
    name=INDEX_NAME,
    fields=[
        SearchField(name="id", type="Edm.String", key=True, filterable=True),
        SearchField(name="parent_id", type="Edm.String", filterable=True),
        SearchField(name="chunk_index", type="Edm.Int32", sortable=True),
        SearchField(name="ntrs_id", type="Edm.String", filterable=True),
        SearchField(name="title", type="Edm.String", searchable=True),
        SearchField(name="year", type="Edm.String", filterable=True, facetable=True),
        SearchField(name="subjects", type="Collection(Edm.String)", filterable=True, facetable=True),
        SearchField(name="content", type="Edm.String", searchable=True),
        SearchField(name="content_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                    vector_search_dimensions=3072, vector_search_profile_name="vector-profile", searchable=True),
    ],
    vector_search=VectorSearch(
        algorithms=[HnswAlgorithmConfiguration(name="hnsw-algo")],
        profiles=[VectorSearchProfile(name="vector-profile", algorithm_configuration_name="hnsw-algo")]
    ),
    semantic_search=SemanticSearch(
        default_configuration_name="semantic-config",
        configurations=[SemanticConfiguration(name="semantic-config", prioritized_fields=SemanticPrioritizedFields(
            content_fields=[SemanticField(field_name="content")], title_field=SemanticField(field_name="title")
        ))]
    )
)

# Retry loop for RBAC propagation
for attempt in range(6):
    try:
        index_client.create_or_update_index(index)
        print(f"✅ Index '{INDEX_NAME}' created!")
        break
    except HttpResponseError as e:
        if 'Forbidden' in str(e) and attempt < 5:
            print(f"   RBAC not ready, waiting 30s... (attempt {attempt+1}/6)")
            time.sleep(30)
        else:
            raise

✅ Index 'apollo-14-research' created!


## Step 9: Generate Embeddings & Upload to Search Index

In [87]:
CHUNK_SIZE = 4000
CHUNK_OVERLAP = 400

def chunk_text(text: str, chunk_size: int = CHUNK_SIZE, overlap: int = CHUNK_OVERLAP) -> list[str]:
    """Split text into chunks at semantic boundaries."""
    if len(text) <= chunk_size:
        return [text]
    
    chunks = []
    start = 0
    
    while start < len(text):
        end = start + chunk_size
        
        if end >= len(text):
            chunks.append(text[start:])
            break
        
        chunk = text[start:end]
        split_pos = chunk.rfind('\n\n')
        if split_pos > chunk_size * 0.5:
            end = start + split_pos + 2
        else:
            for pattern in ['. ', '.\n', '? ', '! ']:
                pos = chunk.rfind(pattern)
                if pos > chunk_size * 0.5:
                    split_pos = pos
                    break
            if split_pos > chunk_size * 0.5:
                end = start + split_pos + 2
            else:
                split_pos = chunk.rfind(' ')
                if split_pos > 0:
                    end = start + split_pos + 1
        
        chunks.append(text[start:end].strip())
        start = end - overlap
    
    return [c for c in chunks if c.strip()]

def get_embedding(text: str, max_chars: int = 8191, max_retries: int = 5) -> list:
    """Get embedding with exponential backoff."""
    for attempt in range(max_retries):
        resp = requests.post(
            f"{APIM_URL}/deployments/{EMBEDDING_MODEL}/embeddings?api-version=2024-10-21",
            headers={"api-key": APIM_KEY, "Content-Type": "application/json"},
            json={"input": text[:max_chars], "model": EMBEDDING_MODEL}
        )
        if resp.status_code == 429:
            wait = (2 ** attempt) + 1
            print(f"⏳ Rate limited, waiting {wait}s...", end=" ", flush=True)
            time.sleep(wait)
            continue
        resp.raise_for_status()
        return resp.json()["data"][0]["embedding"]
    raise Exception("Max retries exceeded")

# Test embedding
test_emb = get_embedding("Apollo 14 lunar mission")
print(f"✅ Embedding model working! Dimension: {len(test_emb)}")

# Generate chunks and embeddings
print(f"\n📄 Chunking {len(documents)} documents...")
search_docs = []
total_chunks = 0

for doc_idx, doc in enumerate(documents):
    chunks = chunk_text(doc['text'])
    print(f"  [{doc_idx+1}/{len(documents)}] {doc['title'][:35]}... → {len(chunks)} chunks")
    
    for chunk_idx, chunk in enumerate(chunks):
        embed_text = f"{doc['title']}\n\n{chunk}"
        
        search_docs.append({
            "id": f"{doc_idx+1}_{chunk_idx}",
            "parent_id": str(doc_idx + 1),
            "chunk_index": chunk_idx,
            "ntrs_id": doc['ntrs_id'],
            "title": doc['title'],
            "year": doc['year'],
            "content": chunk,
            "content_vector": get_embedding(embed_text)
        })
        total_chunks += 1
        time.sleep(0.5)
    print()

print(f"\n✅ Created {total_chunks} chunks from {len(documents)} documents")

# Upload
print(f"\n📤 Uploading to Azure AI Search...")
with SearchIndexingBufferedSender(endpoint=SEARCH_ENDPOINT, index_name=INDEX_NAME, credential=credential) as sender:
    sender.upload_documents(documents=search_docs)

print(f"✅ Uploaded {len(search_docs)} chunks to index '{INDEX_NAME}'!")

✅ Embedding model working! Dimension: 3072

📄 Chunking 8 documents...
  [1/8] Apollo 14 (mission H-3) Baseline Mi... → 7 chunks

  [2/8] Apollo operations handbook, extrave... → 14 chunks

  [3/8] Apollo/Saturn 5 consolidated instru... → 14 chunks

  [4/8] Apollo 14 /AS-509/ operational traj... → 19 chunks

  [5/8] Apollo 14 laser ranging retro-refle... → 13 chunks

  [6/8] Apollo 14 mission, 5 day report... → 14 chunks

  [7/8] Apollo 14 composite casting demonst... → 30 chunks

  [8/8] Apollo 14 lunar photography.  Part ... → 14 chunks


✅ Created 125 chunks from 8 documents

📤 Uploading to Azure AI Search...
✅ Uploaded 125 chunks to index 'apollo-14-research'!


## Step 10: Create Foundry IQ Knowledge Base

In [88]:
# =============================================================================
# FOUNDRY IQ CLIENT
# =============================================================================

class FoundryIQClient:
    """Client for Foundry IQ knowledge bases."""
    
    def __init__(self, search_endpoint: str, get_token):
        self.search_endpoint = search_endpoint.rstrip('/')
        self.get_token = get_token
        self.api_version = "2025-11-01-preview"
    
    def _headers(self):
        return {"Authorization": f"Bearer {self.get_token()}", "Content-Type": "application/json"}
    
    def _parse_response(self, resp):
        """Parse response, handling empty bodies for 2xx responses."""
        if resp.ok:
            if resp.text:
                return resp.json()
            else:
                return {"status": "success", "code": resp.status_code}
        else:
            return {"error": resp.status_code, "message": resp.text}
    
    def create_source(self, name: str, index_name: str, content_fields: list, title_field: str, url_field: str = None):
        """Create a knowledge source pointing to an existing search index."""
        # Build source_data_fields from content_fields and title_field
        source_data_fields = [{"name": f} for f in content_fields]
        if title_field:
            source_data_fields.append({"name": title_field})
        
        body = {
            "name": name,
            "kind": "searchIndex",
            "description": f"Knowledge source for {index_name}",
            "searchIndexParameters": {
                "searchIndexName": index_name,
                "semanticConfigurationName": "semantic-config",
                "sourceDataFields": source_data_fields,
                "searchFields": []
            }
        }
        resp = requests.put(f"{self.search_endpoint}/knowledgesources/{name}?api-version={self.api_version}",
                           headers=self._headers(), json=body)
        return self._parse_response(resp)
    
    def create_kb(self, name: str, sources: list, description: str, apim_url: str, apim_key: str, model: str):
        """Create a knowledge base that references one or more knowledge sources."""
        body = {
            "name": name,
            "knowledgeSources": [{"name": s} for s in sources],
            "description": description,
            "models": [
                {
                    "kind": "azureOpenAI",
                    "azureOpenAIParameters": {
                        "resourceUri": apim_url.replace('/openai', ''),
                        "deploymentId": model,
                        "modelName": model,
                        "apiKey": apim_key
                    }
                }
            ],
            "retrievalReasoningEffort": {"kind": "low"}
        }
        resp = requests.put(f"{self.search_endpoint}/knowledgebases/{name}?api-version={self.api_version}",
                           headers=self._headers(), json=body)
        return self._parse_response(resp)
    
    def query(self, kb_name: str, query: str):
        """Query a knowledge base using the retrieve action."""
        body = {
            "messages": [
                {
                    "role": "user",
                    "content": [{"type": "text", "text": query}]
                }
            ]
        }
        resp = requests.post(f"{self.search_endpoint}/knowledgebases/{kb_name}/retrieve?api-version={self.api_version}",
                            headers=self._headers(), json=body)
        return self._parse_response(resp)
    
    def mcp_url(self, kb_name: str) -> str:
        return f"{self.search_endpoint}/knowledgebases/{kb_name}/mcp?api-version={self.api_version}"


def get_search_token():
    return credential.get_token("https://search.azure.com/.default").token

iq = FoundryIQClient(SEARCH_ENDPOINT, get_search_token)
print("✅ Foundry IQ client initialized")

✅ Foundry IQ client initialized


In [89]:
# Create data source
SOURCE_NAME = f"{INDEX_NAME}-source"
result = iq.create_source(SOURCE_NAME, INDEX_NAME, ["content"], "title", None)
if 'error' not in result:
    print(f"✅ Data source '{SOURCE_NAME}' created!")
else:
    print(f"❌ Error: {result}")

✅ Data source 'apollo-14-research-source' created!


In [ ]:
# Create knowledge base
KB_NAME = f"{INDEX_NAME}-kb"
result = iq.create_kb(KB_NAME, [SOURCE_NAME], f"NASA Technical Reports: {SEARCH_TERM}", APIM_URL, APIM_KEY, MODEL_NAME)

if 'error' not in result:
    print(f"✅ Knowledge base '{KB_NAME}' ready!")
    MCP_URL = iq.mcp_url(KB_NAME)
    print(f"   MCP Endpoint: {MCP_URL[:80]}...")
else:
    print(f"❌ Error: {result}")

In [91]:
# Test knowledge base
test_result = iq.query(KB_NAME, "What scientific instruments did Apollo 14 deploy?")
if 'error' not in test_result:
    # Check for response content
    response = test_result.get('response', [])
    references = test_result.get('references', [])
    print(f"✅ Knowledge base working!")
    print(f"   Response messages: {len(response)}")
    print(f"   References: {len(references)}")
    if response:
        for msg in response:
            content = msg.get('content', [])
            if content:
                text = content[0].get('text', '')[:200]
                print(f"   First response: {text}...")
else:
    print(f"❌ Error: {test_result}")

✅ Knowledge base working!
   Response messages: 1
   References: 1
   First response: [{"ref_id":0,"title":"Apollo 14 laser ranging retro-reflector experiment - Design certification review report","content":"lc:t--. \"r. r 1\nS,\t DY-SICN VERIFICATIONr\n3.\t 1 I_KRR TESY PRO(-,.RAM\n`,...


---

# 🔬 Deep Research with Function Calling

Now we run **o3-deep-research** using **function calling** instead of MCP.

This approach:
1. Uses Azure OpenAI Chat Completions API (which works today)
2. Implements an **agentic loop** that continues until research is complete
3. Provides `search` and `fetch` tools that query Foundry IQ

## Step 11: Define Research Tools

These tools will be called by o3-deep-research during its research process.

In [92]:
import hashlib

# Document cache for fetch operations
_doc_cache: Dict[str, Dict[str, Any]] = {}

# =============================================================================
# TOOL DEFINITIONS (OpenAI Function Calling Schema)
# =============================================================================

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "Search NASA Technical Reports for relevant documents. Returns summaries with IDs that can be fetched for full content.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Natural language search query"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "fetch",
            "description": "Fetch complete document content by ID. Use after search to get full details for citation.",
            "parameters": {
                "type": "object",
                "properties": {
                    "document_id": {
                        "type": "string",
                        "description": "Document ID from search results"
                    }
                },
                "required": ["document_id"]
            }
        }
    }
]

# =============================================================================
# TOOL IMPLEMENTATIONS (Query Foundry IQ)
# =============================================================================

def tool_search(query: str) -> Dict[str, Any]:
    """Search Foundry IQ knowledge base."""
    print(f"   🔍 search('{query[:50]}...')")
    
    result = iq.query(KB_NAME, query)
    
    if 'error' in result:
        return {"error": result['message'], "results": []}
    
    documents = []
    
    # Parse the response from Foundry IQ (2025-11-01-preview format)
    response = result.get('response', [])
    references = result.get('references', [])
    
    # Extract documents from the response content
    for msg in response:
        content = msg.get('content', [])
        for item in content:
            if item.get('type') == 'text':
                try:
                    # The text is a JSON array of documents
                    docs_json = json.loads(item.get('text', '[]'))
                    if isinstance(docs_json, list):
                        for doc in docs_json[:10]:
                            doc_id = str(doc.get('ref_id', hashlib.md5(str(doc).encode()).hexdigest()[:12]))
                            parsed = {
                                "id": doc_id,
                                "title": doc.get('title', 'Untitled'),
                                "text": doc.get('content', '')[:500] + "...",
                                "url": f"https://ntrs.nasa.gov/search"
                            }
                            documents.append(parsed)
                            # Cache full content for fetch
                            _doc_cache[doc_id] = {
                                "id": doc_id,
                                "title": doc.get('title', 'Untitled'),
                                "text": doc.get('content', ''),
                                "url": parsed['url']
                            }
                except json.JSONDecodeError:
                    # If not JSON, treat as plain text
                    pass
    
    print(f"      → Found {len(documents)} documents")
    return {"query": query, "total_results": len(documents), "results": documents}


def tool_fetch(document_id: str) -> Dict[str, Any]:
    """Fetch cached document by ID."""
    print(f"   📄 fetch('{document_id}')")
    
    if document_id not in _doc_cache:
        return {"error": f"Document '{document_id}' not found. Use search first."}
    
    doc = _doc_cache[document_id]
    print(f"      → Fetched: {doc['title'][:40]}...")
    return doc


def execute_tool(name: str, arguments: Dict[str, Any]) -> str:
    """Execute a tool and return JSON result."""
    if name == "search":
        result = tool_search(arguments.get("query", ""))
    elif name == "fetch":
        result = tool_fetch(arguments.get("document_id", ""))
    else:
        result = {"error": f"Unknown tool: {name}"}
    
    return json.dumps(result, indent=2)

print("✅ Research tools defined")
print("   - search: Query Foundry IQ knowledge base")
print("   - fetch: Get full document content for citation")

✅ Research tools defined
   - search: Query Foundry IQ knowledge base
   - fetch: Get full document content for citation


## Step 12: Initialize Azure OpenAI Clients

All API calls go through **APIM gateway** for governance and rate limiting:
- **Deep Research**: o3-deep-research via APIM (routed to Norway East backend)
- **Final Synthesis**: gpt-4.1-mini via APIM (eastus2 hub)

In [ ]:
from openai import AzureOpenAI

# Initialize clients - all calls go through APIM gateway
# Deep research client uses APIM with API key (routes to Norway East backend)
deep_research_client = AzureOpenAI(
    azure_endpoint=APIM_URL.replace('/openai', ''),
    api_key=APIM_KEY,
    api_version="2024-12-01-preview",
    timeout=300
)

# Chat client for final report synthesis (gpt-4.1-mini via APIM)
chat_client = AzureOpenAI(
    azure_endpoint=APIM_URL.replace('/openai', ''),
    api_key=APIM_KEY,
    api_version="2024-10-21",
    timeout=120
)

print(f"✅ Azure OpenAI clients initialized (via APIM gateway)")
print(f"   APIM Gateway: {APIM_URL}")
print(f"   Deep Research Model: {DEEP_RESEARCH_MODEL}")
print(f"   Synthesis Model: {MODEL_NAME}")

## Step 13: Run Deep Research

This runs an **agentic loop** where o3-deep-research:
1. Analyzes the query and plans research steps
2. Calls `search` to find relevant documents
3. Calls `fetch` to get full content for promising results
4. **gpt-4.1-mini** synthesizes findings into a comprehensive final report

> 🔒 All API calls are routed through **APIM gateway** for governance and rate limiting.

In [94]:
@dataclass
class ResearchResult:
    """Results from deep research."""
    query: str
    iterations: int = 0
    tool_calls: List[Dict[str, Any]] = field(default_factory=list)
    final_answer: str = ""
    reasoning_tokens: int = 0
    total_tokens: int = 0
    duration_seconds: float = 0.0
    error: Optional[str] = None


def run_deep_research(query: str) -> ResearchResult:
    """
    Run deep research using agentic loop with function calling.
    """
    result = ResearchResult(query=query)
    start_time = time.time()
    
    # System prompt for research behavior
    system_prompt = f"""You are a deep research assistant with access to NASA Technical Reports about {SEARCH_TERM}.

Your task is to thoroughly research the user's query by:
1. Use the 'search' tool to find relevant documents in the knowledge base
2. Use the 'fetch' tool to get full content of the most relevant documents
3. Analyze and synthesize the information
4. Provide a comprehensive, well-cited answer

IMPORTANT GUIDELINES:
- Search multiple times with different queries to get comprehensive coverage
- Fetch all documents that seem relevant before writing your final answer
- Include specific facts, dates, figures, and technical details from the documents
- Cite your sources using document IDs (e.g., [doc-abc123])
- Structure your final answer with clear sections and headers
- Be thorough - this is deep research, not a quick summary

When you have gathered enough information, provide your final research report."""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query}
    ]
    
    print(f"\n{'='*60}")
    print(f"🔬 DEEP RESEARCH STARTED")
    print(f"{'='*60}")
    print(f"Query: {query[:80]}...")
    print()
    
    try:
        for iteration in range(MAX_RESEARCH_ITERATIONS):
            print(f"\n📍 Iteration {iteration + 1}/{MAX_RESEARCH_ITERATIONS}")
            
            # Call the model
            response = deep_research_client.chat.completions.create(
                model=DEEP_RESEARCH_MODEL,
                messages=messages,
                tools=TOOLS
            )
            
            message = response.choices[0].message
            result.total_tokens += response.usage.total_tokens if response.usage else 0
            
            # Check for reasoning tokens (o3 specific)
            if response.usage and hasattr(response.usage, 'completion_tokens_details'):
                details = response.usage.completion_tokens_details
                if details and hasattr(details, 'reasoning_tokens'):
                    result.reasoning_tokens += details.reasoning_tokens or 0
            
            # If no tool calls, we have gathered info - synthesize with gpt-4.1-mini
            if not message.tool_calls:
                print("\n✅ Research complete, synthesizing final report with gpt-4.1-mini...")
                
                # Gather all research context for synthesis
                research_context = message.content or ""
                
                # Create synthesis prompt with all gathered information
                synthesis_messages = [
                    {"role": "system", "content": f"""You are an expert research report writer. 
Based on the research conducted by the deep research model, write a comprehensive, well-structured final report.

GUIDELINES:
- Structure the report with clear headers and sections
- Include specific facts, dates, figures, and technical details
- Cite sources using document IDs where available (e.g., [doc-abc123])
- Be thorough and comprehensive
- Use professional academic writing style"""},
                    {"role": "user", "content": f"""Based on this research about {SEARCH_TERM}, write a comprehensive final report:

ORIGINAL QUERY:
{query}

RESEARCH FINDINGS:
{research_context}

Please synthesize this into a well-organized, comprehensive research report."""}
                ]
                
                # Use gpt-4.1-mini for final synthesis
                synthesis_response = chat_client.chat.completions.create(
                    model=MODEL_NAME,
                    messages=synthesis_messages
                )
                
                result.final_answer = synthesis_response.choices[0].message.content or ""
                if synthesis_response.usage:
                    result.total_tokens += synthesis_response.usage.total_tokens
                result.iterations = iteration + 1
                break
            
            # Process tool calls
            messages.append(message)
            
            for tool_call in message.tool_calls[:5]:  # Limit per iteration
                func_name = tool_call.function.name
                func_args = json.loads(tool_call.function.arguments)
                
                # Execute tool
                tool_result = execute_tool(func_name, func_args)
                
                # Record tool call
                result.tool_calls.append({
                    "iteration": iteration + 1,
                    "tool": func_name,
                    "arguments": func_args
                })
                
                # Add tool response to messages
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": tool_result
                })
            
            # Check if max iterations reached
            if iteration == MAX_RESEARCH_ITERATIONS - 1:
                print("\n⚠️ Max iterations reached, synthesizing final report with gpt-4.1-mini...")
                
                # Gather all tool call results for synthesis
                # Handle both dict messages and ChatCompletionMessage objects
                gathered_info = []
                for msg in messages:
                    if isinstance(msg, dict):
                        if msg.get("role") == "tool":
                            gathered_info.append(msg.get("content", ""))
                    # Skip ChatCompletionMessage objects (they don't have tool results)
                
                # Create synthesis prompt
                synthesis_messages = [
                    {"role": "system", "content": f"""You are an expert research report writer.
Based on the research data gathered about {SEARCH_TERM}, write a comprehensive, well-structured final report.

GUIDELINES:
- Structure the report with clear headers and sections
- Include specific facts, dates, figures, and technical details
- Cite sources using document IDs where available (e.g., [doc-abc123])
- Be thorough and comprehensive
- Use professional academic writing style"""},
                    {"role": "user", "content": f"""Based on this research about {SEARCH_TERM}, write a comprehensive final report:

ORIGINAL QUERY:
{query}

GATHERED RESEARCH DATA:
{chr(10).join(gathered_info[:5])}

Please synthesize this into a well-organized, comprehensive research report."""}
                ]
                
                # Use gpt-4.1-mini for final synthesis
                final_response = chat_client.chat.completions.create(
                    model=MODEL_NAME,
                    messages=synthesis_messages
                )
                result.final_answer = final_response.choices[0].message.content or ""
                result.iterations = iteration + 1
                if final_response.usage:
                    result.total_tokens += final_response.usage.total_tokens
        
    except Exception as e:
        result.error = str(e)
        print(f"\n❌ Error: {e}")
    
    result.duration_seconds = round(time.time() - start_time, 2)
    
    print(f"\n{'='*60}")
    print(f"🔬 DEEP RESEARCH COMPLETE")
    print(f"{'='*60}")
    print(f"   Iterations:       {result.iterations}")
    print(f"   Tool calls:       {len(result.tool_calls)}")
    print(f"   Total tokens:     {result.total_tokens:,}")
    print(f"   Reasoning tokens: {result.reasoning_tokens:,}")
    print(f"   Duration:         {result.duration_seconds}s")
    
    return result

print("✅ Deep research function ready")

✅ Deep research function ready


## Step 14: Execute Deep Research Query

In [95]:
# Clear document cache for fresh research
_doc_cache.clear()

# Define research query
research_query = f"""Research {SEARCH_TERM} comprehensively. 
I need to understand:
1. What was special about this mission?
2. What scientific instruments were deployed?
3. What were the key findings from the lunar samples?
4. Who was the commander and what made him notable?

Provide a detailed research report with specific facts, dates, and citations."""

# Run deep research
research_result = run_deep_research(research_query)


🔬 DEEP RESEARCH STARTED
Query: Research Apollo 14 comprehensively. 
I need to understand:
1. What was special a...


📍 Iteration 1/10
   🔍 search('Apollo 14 mission overview NASA Technical Reports...')
      → Found 5 documents

📍 Iteration 2/10
   📄 fetch('0')
      → Fetched: Apollo 14 mission, 5 day report...

📍 Iteration 3/10
   📄 fetch('1971027929')

📍 Iteration 4/10
   🔍 search('"Apollo 14 mission, 5 day report" 1971027929...')
      → Found 0 documents

📍 Iteration 5/10
   🔍 search('Apollo 14 scientific instruments deployed ALSEP...')
      → Found 3 documents

📍 Iteration 6/10
   🔍 search('"Apollo 14 Preliminary Science Report"...')
      → Found 6 documents

📍 Iteration 7/10
   🔍 search('Apollo 14 Preliminary Science Report lunar samples...')
      → Found 5 documents

📍 Iteration 8/10
   🔍 search('"Apollo 14 Mission Report" NASA 1971...')
      → Found 1 documents

📍 Iteration 9/10
   📄 fetch('0')
      → Fetched: Apollo 14 mission, 5 day report...

📍 Iteration 10/10
   📄 fe

## Step 15: Display Research Report

In [96]:
# Display the research report
if research_result.error:
    display(HTML(f'<div style="background:#ffdddd;padding:15px;border-radius:8px;">'
                 f'<h3>❌ Research Error</h3><p>{research_result.error}</p></div>'))
else:
    # Summary card
    html = f'''
    <div style="font-family: system-ui; padding: 20px; background: linear-gradient(135deg, #1a1a2e, #16213e); 
                border-radius: 12px; margin: 10px 0;">
        <h2 style="color: #4da6ff; margin: 0 0 15px 0;">🔬 Deep Research Results</h2>
        <div style="display: flex; gap: 20px; flex-wrap: wrap;">
            <div style="background: rgba(15,52,96,0.3); padding: 12px 20px; border-radius: 8px; text-align: center;">
                <div style="font-size: 24px; color: #4da6ff; font-weight: bold;">{research_result.iterations}</div>
                <div style="color: #888; font-size: 12px;">Iterations</div>
            </div>
            <div style="background: rgba(15,52,96,0.3); padding: 12px 20px; border-radius: 8px; text-align: center;">
                <div style="font-size: 24px; color: #28a745; font-weight: bold;">{len(research_result.tool_calls)}</div>
                <div style="color: #888; font-size: 12px;">Tool Calls</div>
            </div>
            <div style="background: rgba(15,52,96,0.3); padding: 12px 20px; border-radius: 8px; text-align: center;">
                <div style="font-size: 24px; color: #ffc107; font-weight: bold;">{research_result.total_tokens:,}</div>
                <div style="color: #888; font-size: 12px;">Total Tokens</div>
            </div>
            <div style="background: rgba(15,52,96,0.3); padding: 12px 20px; border-radius: 8px; text-align: center;">
                <div style="font-size: 24px; color: #e94560; font-weight: bold;">{research_result.duration_seconds}s</div>
                <div style="color: #888; font-size: 12px;">Duration</div>
            </div>
        </div>
    </div>
    '''
    display(HTML(html))
    
    # Research report
    display(Markdown("---\n## 📄 Research Report\n\n" + research_result.final_answer))

---
## 📄 Research Report

**Final Research Report on Apollo 14 Mission**

---

**1. Introduction**

Apollo 14, the eighth crewed mission in NASA's Apollo program and the third to land humans on the Moon, holds a distinguished place in the annals of lunar exploration. Launched on January 31, 1971, and lasting five days, Apollo 14 was pivotal in continuing scientific exploration of the lunar surface following the technical difficulties of earlier missions. This report provides a comprehensive overview of the mission’s unique features, scientific instrumentation deployed, key findings from lunar samples, and details about its notable commander.

---

**2. Unique Aspects and Significance of Apollo 14**

Apollo 14 was special primarily for overcoming technical challenges and successfully conducting targeted scientific and exploratory objectives. This mission specifically aimed to explore the Fra Mauro formation, a site of high scientific interest due to its geologic complexity and as the intended landing site for the aborted Apollo 13 mission.

Key characteristics of the mission included:

- **Targeted Landing:** The lunar module performed a powered descent and landing over revolution 14 in lunar orbit, utilizing descent targeting strategies refined from Apollo 12, adjusting for the challenging terrain at Fra Mauro [doc-1].

- **Extended Surface Activities:** Crew executed a series of extravehicular activities (EVAs) involving surface experiments and sample collection. Despite a minor delay in cabin depressurization by roughly 10 minutes, the astronauts succeeded in deploying equipment and collecting extensive scientific data [doc-3].

- **Enhanced Mobility and Sampling:** Compared to prior missions, Apollo 14's astronauts noted differences in lunar soil mechanics, including deeper footpad penetration of the lunar module and notable dust behavior during descent below 100 feet altitude [doc-0].

The mission recovered critical data to advance lunar geology understanding and refined EVA and equipment deployment procedures in the lunar environment.

---

**3. Scientific Instruments Deployed**

Apollo 14 deployed an array of scientific instruments primarily comprising the Apollo Lunar Surface Experiments Package (ALSEP) and additional specific devices aimed at lunar environmental and geophysical measurements:

- **Laser Ranging Retro-reflector (LRRR):** Similar in design to that deployed during Apollo 11, this device reflected laser signals back to Earth, facilitating precise measurements of the Earth-Moon distance by timing the reflection delay [doc-0][doc-0 alt].

- **Solar Wind Composition Experiment:** Exposed on the lunar surface for approximately 21 hours, this experiment collected particles emitted by the Sun to analyze the solar wind's elemental and isotopic composition [doc-0].

- **Lunar Portable Magnetometer:** This instrument gathered magnetic field data at two separate sites on the lunar surface, yielding valuable insight into the Moon's magnetic environment [doc-0].

- **Seismic and Soil Mechanics Tools:** Including a 26.5-inch geophone cable anchor used to probe soil penetration resistance and a trench excavation facilitating stratigraphic sampling of lunar regolith layers [doc-0].

- **Communications and Telemetry Systems:** Comprehensive telemetry across multiple frequency bands alongside onboard television equipment transmitted real-time color television of surface operations [doc-2][doc-3].

Overall, these instruments were critical for characterizing the lunar environment, surface geophysics, and delivering unprecedented insights into solar-lunar interactions.

---

**4. Key Findings from Lunar Samples**

Apollo 14's sampling activities yielded a diverse collection of regolith and rock materials, revealing new details about lunar geology:

- **Soil Mechanics and Stratigraphy:** Investigations exposed three distinct layers near North Triplet Crater:  
  1. A dark brown, fine-grained surface layer  
  2. An intermediate thin layer predominantly composed of glassy patches  
  3. A very light-colored granular material at depth [doc-0]

- **Surface Soil Characteristics:** Dust behavior was observed during descent, with lunar soil adhering extensively to clothing and equipment, and footpad penetration depth exceeding previous missions, indicating varied mechanical properties at the Fra Mauro site [doc-0].

- **Trench Excavation:** Sampling to a depth of 18 inches revealed mechanically distinct regolith horizons and elucidated the subsurface structure and layering. Notably, collapsing trench walls limited deeper excavation [doc-0].

- **Lunar Soil Instrumentation Engagement:** Tests of soil penetration resistance via the geophone cable anchor confirmed variations in mechanical strength, a factor critical for future surface exploration and construction considerations [doc-0].

These findings underscored notable geological diversity at the Fra Mauro site compared to earlier Apollo landing locations and contributed to refining models of lunar soil properties.

---

**5. Apollo 14 Commander: Alan Shepard**

Commander Alan B. Shepard Jr. was a defining figure of Apollo 14’s success and distinction:

- **Historical Notability:** Shepard was the first American astronaut to travel into space in 1961 aboard Mercury-Redstone 3 (Freedom 7), marking the United States' breakthrough in human spaceflight.

- **Return to Flight:** After a medical condition grounded him for years, Shepard was fully restored to flight status, making Apollo 14 his triumphant return to space.

- **Skills and Contributions:** Beyond command duties, Shepard famously performed the first golf shots on the Moon, bringing a human and light-hearted element to lunar exploration while demonstrating crew adaptability [doc-3].

- **Leadership:** His experience and calm demeanor helped overcome mission delays and ensured smooth execution of surface EVA tasks and equipment deployment.

Shepard remains a celebrated astronaut, underpinning Apollo 14 with both technical expertise and historical significance.

---

**6. Conclusion**

Apollo 14 was a landmark mission that exemplified NASA’s resilience and scientific ambition during the Apollo era. It combined meticulous planning, advanced instrumentation deployment, and skilled astronaut leadership to achieve vital exploration and research milestones on the Moon. The comprehensive scientific data and lunar samples gathered expanded knowledge of the Moon’s geology, environment, and space weather influences. Commander Alan Shepard’s pivotal role marked a triumphant chapter in human spaceflight history.

---

**References**

- Apollo 14 Mission, 5 Day Report, NASA Technical Reports [doc-0], [doc-3], [doc-4]  
- Apollo 14 (mission H-3) Baseline Mission Profile, NASA Technical Reports [doc-1]  
- Apollo/Saturn 5 Consolidated Instrumentation Plan for AS-509 /Apollo 14/, NASA Technical Reports [doc-2]  
- Apollo 14 Laser Ranging Retro-reflector Experiment - Design Certification Review Report, NASA Technical Reports [doc-0 alt]

*Note: Document identifiers correspond to source data indexed from NASA Technical Report Server (NTRS).*

---

**Prepared by: [Your Name]  
Date: [Current Date]**

## Step 16: Analyze Tool Usage

In [97]:
# Show tool call breakdown
if research_result.tool_calls:
    display(Markdown("### 🔧 Tool Call Summary"))
    
    search_calls = [t for t in research_result.tool_calls if t['tool'] == 'search']
    fetch_calls = [t for t in research_result.tool_calls if t['tool'] == 'fetch']
    
    print(f"Search calls: {len(search_calls)}")
    print(f"Fetch calls:  {len(fetch_calls)}")
    print()
    
    print("Search queries:")
    for i, call in enumerate(search_calls, 1):
        query = call['arguments'].get('query', 'N/A')
        print(f"  {i}. {query[:70]}...")
    
    print(f"\nDocuments fetched: {len(fetch_calls)}")
else:
    print("No tool calls recorded.")

### 🔧 Tool Call Summary

Search calls: 6
Fetch calls:  4

Search queries:
  1. Apollo 14 mission overview NASA Technical Reports...
  2. "Apollo 14 mission, 5 day report" 1971027929...
  3. Apollo 14 scientific instruments deployed ALSEP...
  4. "Apollo 14 Preliminary Science Report"...
  5. Apollo 14 Preliminary Science Report lunar samples analysis findings...
  6. "Apollo 14 Mission Report" NASA 1971...

Documents fetched: 4


---

## Summary

You've completed the **Deep Research** lab using:

| Component | Description |
|-----------|-------------|
| **o3-deep-research** | Advanced reasoning model with multi-step tool use |
| **gpt-4.1-mini** | Final report synthesis (cost-effective) |
| **APIM Gateway** | All API calls routed through gateway for governance |
| **Foundry IQ** | Knowledge base for NASA Technical Reports |
| **Agentic Loop** | Iterative search → fetch → synthesize pattern |

### Architecture Benefits

- ✅ **All calls via APIM** - centralized governance, rate limiting, and observability
- ✅ **Multi-region Landing Zone** pattern (Norway East for o3-deep-research)
- ✅ **Cost optimization** - o3-deep-research for reasoning, gpt-4.1-mini for synthesis
- ✅ **Foundry IQ integration** for enterprise knowledge bases
- ✅ **Citation-rich reports** with source tracking

### Configuration

| Setting | Value |
|---------|-------|
| MAX_RESEARCH_ITERATIONS | 10 |
| Deep Research Model | o3-deep-research |
| Synthesis Model | gpt-4.1-mini |

### Next Steps

- Try different `SEARCH_TERM` values ("Mars", "Voyager", "Space Shuttle")
- Adjust `MAX_RESEARCH_ITERATIONS` for more/less thorough research
- Add more tools (web search, code execution)
- Monitor usage via APIM analytics

## Cleanup (Optional)

In [98]:
# Uncomment to delete resources
# !az group delete -n "{RG}" --yes --no-wait
# print("✅ Cleanup initiated")